In [1]:
os.getcwd()

NameError: name 'os' is not defined

In [6]:
#Counts QA

#set up sql alchemy engine
import pandas as pd
import os
import configparser
import sys
sys.path.append('/project/Development')
from helpers import general_helpers
import configparser
config = configparser.ConfigParser()
config.read('/project/Development/config.ini')
host = config['DATABASE']['HOST']
username = config['DATABASE']['USERNAME']
password = config['DATABASE']['PASSWORD']
new_database = config['DATABASE']['NEW_DB']
old_database = config['DATABASE']['OLD_DB']
temporary_upload = config['DATABASE']['TEMP_UPLOAD_DB']
previous_qa_loc = config['FOLDERS']['OLD_QA_LOC']
new_qa_loc = config['FOLDERS']['OLD_QA_LOC']
latest_expected_date = config['CONSTANTS']['LATEST_DATE']

In [9]:
os.listdir("/project/previous_QA_output")

['10_db_parameters.xlsx',
 '1_distinct_to_total.xlsx',
 '1_latest_date_check.xlsx',
 '1_null_and_blank_count.xlsx',
 '1_table_counts.xlsx',
 '2_data_by_year.xlsx',
 '3_right_joins_nonstandard.xlsx',
 '3_right_joins_overnight_1.xlsx',
 '3_right_joins_standard.xlsx',
 '4_left_joins.csv',
 '~$4_left_joins_nonstandard_v2.xlsx',
 '4_left_joins_nonstandard_v2.xlsx',
 '4_left_joins_nonstandard.xlsx',
 '4_left_joins_overnight.xlsx',
 '4_left_joins_weird.csv',
 '4_left_joins.xlsx',
 '~$5_loc_distinct.xlsx',
 '5_loc_distinct.xlsx',
 '5_loc_NOT_distinct_test.xlsx',
 '5_loc_NOT_distinct.xlsx',
 '5_right_joins_overnight.xlsx',
 '6_list_length',
 '6_list_length_cd.xlsx',
 '6_list_length.xlsx',
 '7_date_sum_count_distinct_TEST.xlsx',
 '7_date_sum_count_distinct.xlsx',
 '7_date_sum_count_TEST.xlsx',
 '7_date_sum_count.xlsx',
 '8_other_sum_qa_new.xlsx',
 '8_other_sum_qa_null_titles.xlsx',
 '8_other_sum_qa_TEST.xlsx',
 '8_other_sum_qa.xlsx',
 '9_db_parameters.xlsx',
 'date_sum_qa_1_v2.xlsx',
 '~$date_su

In [7]:
engine = general_helpers.connect_to_db(host, username, password, new_database)
data = pd.read_excel("{}/1_table_counts.xlsx".format(previous_qa_loc))
#print (data.head())

In [8]:
#Counts- creating count description
def create_count_description(old_db_count, new_db_count, table, min_inc = 1.01, max_inc = 1.1):
    unchanging = ['cpc_subsection', 'wipo_field', 'nber_category', 'nber', 'cpc_group',
                  'nber_subcategory', 'mainclass', 'mainclass_current', 'subclass', 'subclass_current']
    slight_changes = ['cpc_subgroup']
    if not table in unchanging + slight_changes:
        if new_db_count < old_db_count:
            return "Problem: New table has fewer rows than old table "
        elif new_db_count == old_db_count:
            return "Problem: No new entries"
        elif new_db_count > old_db_count*max_inc:
            return "Problem: Too many new entries"
        elif new_db_count < old_db_count*min_inc:
            return "Problem: Too few new entries"
        elif old_db_count*min_inc < new_db_count < old_db_count*max_inc:
            return "No Problem!"
        else:
            return "Check the logic!"
    elif table in unchanging:
        if new_db_count == old_db_count:
            return "No Problem!"
        else:
            return "Problem: Number of rows in unchanging table changed"
    elif table in slight_changes:
        if old_db_count*.9 < new_db_count < old_db_count*1.1:
            return "No Problem!"
        else: 
            return "Problem: Number of rows in slightly changing table changed"

In [9]:
#Counts- Get counts
def get_counts(previous_qa_loc, new_qa_loc, new_database):
    counts = pd.read_excel('{}/1_table_counts.xlsx'.format(previous_qa_loc))
    engine.execute('use {}'.format(new_database))
    new_counts = []
    for table in counts['Table']:
       # print table
        count = engine.execute('select count(*) from {}'.format(table))
        new_counts.append(count)
    return new_counts
#Counts- write to excel
def make_excel(new_counts, previous_qa_loc, new_qa_loc, new_database):
    counts = pd.read_excel('{}/1_table_counts.xlsx'.format(OLD_QA))
    counts[new_database] = new_counts
    del counts['Description']
    #the last row of the table is now the most recent previous database!
    counts['Description'] = counts.apply(lambda row: create_count_description(row[counts.columns[-3]], row[new_database],row['Table']), axis=1)
    counts.to_excel('{}/1_table_counts.xlsx'.format(new_qa_loc), index = False)

In [ ]:

#Approximately 
new_counts = get_counts(previous_qa_loc, new_qa_loc, new_database)
make_excel(new_counts,previous_qa_loc, new_qa_loc, new_database)
